In [ ]:
import torchvision.transforms.functional as TF

from models import  caption
from datasets import coco
from configuration import Config
from engine import evaluate

from PIL import Image
import numpy as np
import random
import os
import json

import torch
from configuration import Config
from transformers import BertTokenizer

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from models import caption
from datasets import coco, utils


In [ ]:
location = '..\\Bangla Dataset'

## Test the model 

In [ ]:
checkpoint_location = '../checkpoint.pth'

In [ ]:
model, criterion = caption.build_model(Config())
model.load_state_dict(torch.load(checkpoint_location)['model'])

In [ ]:
image_path = '../Bangla Dataset/images/54.png'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

img = Image.open(image_path)
image = coco.val_transform(img)
image = image.unsqueeze(0)

def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template


caption, cap_mask = create_caption_and_mask(
    start_token, Config().max_position_embeddings)

@torch.no_grad()
def evaluate():
    model.eval()
    for i in range(Config().max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption


output = evaluate()
result = tokenizer.decode(output[0].tolist(),skip_special_tokens=True)
#result = tokenizer.decode(output[0], skip_special_tokens=True)
print(result)

display(img)